# 用户的重合度方案

In [ ]:
import pandas as pd
import numpy as np
data = pd.read_csv("C:\\Users\\work\\Desktop\\tencent\\pre\\user_installedapps.csv")
data.head()

# 1.生成app_coScore和app_coScore_log、app_coScore_logReci

In [ ]:
# 读取appScale
readAppScale = pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\app-user\\app_userScale.csv')

In [ ]:
import math
readAppScale['coScale'] = readAppScale['scale'].apply(lambda x: pow(x, 2))
readAppScale['coScale_log'] = readAppScale['scale'].apply(lambda x: math.log(pow(x, 2)))
readAppScale['coScale_logReci'] = readAppScale['coScale_log'].apply(lambda x: 1/(x+0.0001))

In [ ]:
readAppScale.to_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\app-user\\app_coScale.csv', index=False)

# 2. 对于某个targetAPP，生成网络

In [ ]:
readAppCoScale = pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\app-user\\app_coScale.csv')[['appID', 'coScale_logReci']].set_index('appID')
dictAppScore = readAppCoScale.to_dict() #索引，为了加快速度

In [ ]:
# 取某个appID的用户，分为1和0两个部分
# 83大概有1w条记录，拿它进行试验
dataNet  = pd.merge(pd.read_csv("C:\\Users\\work\\Desktop\\tencent\\pre\\train.csv"), pd.read_csv("C:\\Users\\work\\Desktop\\tencent\\pre\\ad.csv"), on='creativeID')
dataNet = dataNet[dataNet['appID']==83][['label','userID']]
dataNet_1 = dataNet[dataNet['label']==1]
dataNet_0 = dataNet[dataNet['label']==0]

In [ ]:
# 读取user_apps
readUserApps = pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\app-user\\user_apps.csv', names=['userID', 'appList'])
dict_userApps = {}
def getDictUserApps(x):
     dict_userApps[x[0]] = x[1][1:-1].split(', ')
readUserApps.apply(getDictUserApps, axis=1)
readUserApps.head()

In [ ]:
# 生成网络
## 前期准备
listUsers1 = list(dataNet_1['userID'])
sampler = np.random.permutation(len(listUsers1))
index = dataNet_0['userID'].index[sampler] #重排序后的索引
listUsers0 = list(dataNet_0['userID'].ix[index])
## 生成id
dict_user_nodeID = {}
nodeID = 0
for i in range(len(listUsers1)):
    nodeID += 1
    dict_user_nodeID[listUsers1[i]] = nodeID
for i in range(len(listUsers0)):
    nodeID += 1
    dict_user_nodeID[listUsers0[i]] = nodeID
## 计算用函数
def getScore(list):
    score = 0
    for l in list:
        score += dictAppScore['coScale_logReci'][int(l)]
    return score
def cal_coScale(user0, user1):
    list0 = dict_userApps[user0]
    list1 = dict_userApps[user1]
    interset = np.intersect1d(list0, list1)
    length = len(interset)
    if(length == 0): 
        return 0
    else: 
#         return getScore(interset)
#         return getScore(interset)/length
        return getScore(np.intersect1d(list0, list1))/getScore(np.union1d(list0, list1))

In [ ]:
# node
## node1
f_node = open('C:\\Users\\work\\Desktop\\tencent\\feature\\net\\83_node.csv', 'w')
f_node.write('Id,Label,partition\r\n')
for i in range(len(listUsers1)):
    f_node.write(str(listUsers1[i]) + ',' + str(listUsers1[i]) + ',' + '1' + '\r\n')
## node0
for i in range(len(listUsers0)):
    f_node.write(str(listUsers0[i]) + ',' + str(listUsers0[i]) + ',' + '0' + '\r\n')
f_node.close()

In [ ]:
# line
def getLine():
    f_line = open('C:\\Users\\work\\Desktop\\tencent\\feature\\net\\83_line.csv', 'w')
    f_line.write('Source,Target,weight,partition\r\n')
    threshold_score = 0.3
    # 1网络
    for i in range(len(listUsers1)):
        if dict_userApps.has_key(listUsers1[i]) == False: continue
        for j in range(i, len(listUsers1)):
            if dict_userApps.has_key(listUsers1[j]) == False: continue
            score = cal_coScale(listUsers1[i], listUsers1[j])
            if score > threshold_score:
                f_line.write(str(listUsers1[i]) + "," + str(listUsers1[j]) + ","
                             + str(score) + ',' + "1\r\n")
    # 0网络
    for i in range(len(listUsers0)):
        if dict_userApps.has_key(listUsers0[i]) == False: continue
        for j in range(i, len(listUsers0)):
            if dict_userApps.has_key(listUsers0[j]) == False: continue
            score = cal_coScale(listUsers0[i], listUsers0[j])
            if score > threshold_score:
                f_line.write(str(listUsers0[i]) + "," + str(listUsers0[j]) + ","
                             + str(score) + ',' + "0\r\n")
    # 1-0网络
    for i in range(len(listUsers1)):
        if dict_userApps.has_key(listUsers1[i]) == False: continue
        for j in range(i, len(listUsers0)):
            if dict_userApps.has_key(listUsers0[j]) == False: continue
            score = cal_coScale(listUsers1[i], listUsers0[j])
            if score > threshold_score:
                f_line.write(str(listUsers1[i]) + "," + str(listUsers0[j]) + ","
                             + str(score) + ',' + "10\r\n")
    f_line.close()

In [ ]:
import timeit
timeit.timeit(getLine,number=1)

In [ ]:
temp = pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\net\\83_line.csv')

In [ ]:
import matplotlib.pyplot as plt
temp.sort_values(by=['weight'], ascending=False, inplace=True)
temp2 = temp['weight']
plt.plot(range(len(temp2)),temp2)
plt.show()
# temp['weight']

# 2.1 求出具有代表性的正负点

In [ ]:
# 点-加权度计算
## 参数
threshold_score = 0.3
## 点-加权度计算函数
def getNodeWeight(listU):
    dict = {}
    for i in range(len(listU)):
        if dict_userApps.has_key(listU[i]) == False: continue
        sum = 0.0
        for j in range(i, len(listU)):
            if dict_userApps.has_key(listU[j]) == False: continue
            score = cal_coScale(listU[i], listU[j])
            if score > threshold_score:
                sum += score
        dict[listU[i]] = sum
    result = pd.DataFrame.from_dict(dict, orient='index')
    result = result.reset_index().sort_values(by=[0],ascending=False)
    return result
## 加权网络
nodeWeight1 = getNodeWeight(listUsers1)
nodeWeight0 = getNodeWeight(listUsers0)

In [ ]:
# 取具有代表性的topN个点
percent = 0.50
repre1 = nodeWeight1[int(nodeWeight1.count()['index']*0.15): int(nodeWeight1.count()['index']*percent):3]
repre0 = nodeWeight0[int(nodeWeight1.count()['index']*0.15): int(nodeWeight0.count()['index']*percent):3]

In [ ]:
# 存入csv
outRepresent1 = pd.DataFrame({'appID':83,'represent':repre1['index'].values})
outRepresent0 = pd.DataFrame({'appID':83,'represent':repre0['index'].values})
outRepresent1.to_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\app-user\\app_representUser_1.csv', index=False)
outRepresent0.to_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\app-user\\app_representUser_0.csv', index=False)

# 2.2 计算用户与1、0代表用户的相似度

In [ ]:
# 读取app的代表用户
readRepresent1 = pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\app-user\\app_representUser_1.csv')
readRepresent0 = pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\app-user\\app_representUser_0.csv')

In [ ]:
# 读取app的代表用户（用gephi生成的）
raw = pd.read_csv("C:\\Users\\work\\Desktop\\tencent\\feature\\net\\nodes.csv")
raw = raw[raw['degree']>51]
raw = raw[raw['degree']<295]
readRepresent1 = raw[raw['partition']==1]['id']
readRepresent0 = raw[raw['partition']==0]['id']

In [ ]:
# 读取app的代表用户（全网络）
raw = pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\net\\node4.csv')[['id', 'partition']]
readRepresent1 = raw[raw['partition']==1]['id']
readRepresent0 = raw[raw['partition']==0]['id']

In [ ]:
seRepresent1[83] = list(readRepresent1)
seRepresent0[83] = list(readRepresent0)

In [ ]:
# 将app-代表用户写入series，便于查询
def getSeries(df):
    return list(df['represent'])
seRepresent1 = readRepresent1.groupby('appID').apply(getSeries)
seRepresent0 = readRepresent0.groupby('appID').apply(getSeries)

In [ ]:
# # 读入训练数据
data = pd.read_csv("C:\\Users\\work\\Desktop\\tencent\\pre\\train.csv")
data = pd.merge(data, pd.read_csv("C:\\Users\\work\\Desktop\\tencent\\pre\\ad.csv"), on='creativeID')
data = data[data['appID']==83][['userID','appID']]

# data = pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\net\\node4.csv')
# data = data[['id']]
# data.rename(columns={'id':'userID'},inplace=True)
# data['appID'] = 83

In [ ]:
threshold_score = 0.3
def getVoteP(x):
    sum = 0.0
    count = 0
    for id in seRepresent1[x[1]]:
        if dict_userApps.has_key(id) == False: #无数据的情况
            continue
        if dict_userApps.has_key(x[0]) == False: #无数据的情况
            continue
        score = cal_coScale(x[0], id)
        if score > threshold_score:
            count += 1
            sum += score
    if count == 0:
        return 0
    else:
        return sum/count
def getVoteN(x):
    sum = 0.0
    count = 0
    for id in seRepresent0[x[1]]:
        if dict_userApps.has_key(id) == False: #无数据的情况
            continue
        if dict_userApps.has_key(x[0]) == False: #无数据的情况
            continue
        score = cal_coScale(x[0], id)
        if score > threshold_score:
            count += 1
            sum += score
    if count == 0:
        return 0
    else:
        return sum/count
def getMinus(x):
    minus = x[0] - x[1]
    sum = x[0] + x[1]
    if minus == 0:
        return 0
    else:
        return float(minus)/sum
def getV(x):
    if x[0] > 0:
        return 1
    elif x[0] < 0: 
        return -1
    else:
        return 0
def getVote():
    data['voteP'] = data[['userID','appID']].apply(getVoteP, axis=1)
    data['voteN'] = data[['userID','appID']].apply(getVoteN, axis=1)
    data['voteMinus'] = data[['voteP', 'voteN']].apply(getVoteMinus, axis=1)
    data['v'] = data[['voteMinus']].apply(getV, axis=1)

In [ ]:
import timeit
timeit.timeit(getVote, number=1)

In [ ]:
data[data['v']!=0]

In [ ]:
# 输出
data.to_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\user_vote.csv',index=False)

In [ ]:
temp = pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\user_vote.csv')

In [ ]:
temp[temp['userID']==2642481]